In [1]:
import tensorflow
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense

import PIL
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

import sklearn
import pandas
import numpy
import json
import os
from sklearn.model_selection import train_test_split

In [2]:
%load_ext tensorboard

In [3]:
image_width = 128
image_height = 128
image_channels = 3
batch_size = 32
epochs = 10

In [4]:
model = tensorflow.keras.Sequential()

In [5]:
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (image_width, image_height, image_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [6]:
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [7]:
model.add(Conv2D(150, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [8]:
model.add(Flatten())

In [9]:
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [10]:
model.add(Dense(2, activation = 'softmax'))

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0

In [13]:
class CustomCallback(tensorflow.keras.callbacks.Callback):
    
    def on_train_begin(self, logs = None):
        print("Starting training on this.date")
        
    def on_train_end(self, logs = None):
        print("Stopped training")
        
    def on_epoch_begin(self, epoch, logs = None):
        print("Epoch begin")
        
    def on_epoch_end(self, epoch, logs = None):
        print("Epoch end")
    
    def on_test_begin(self, logs = None):
        print("Test start")
        
    def on_test_end(self, logs = None):
        print("Test end")
        
    def on_predict_begin(self, logs = None):
        print("Predicting")
        
    def on_predict_end(self, logs = None):
        print("End predicting")
    
    def on_train_batch_begin(self, batch, logs=None):
        print("Train batch begin")

    def on_train_batch_end(self, batch, logs=None):
        print("Train batch end")

    def on_test_batch_begin(self, batch, logs=None):
        print("Test batch begin")

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("Test batch end")

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("Predict batch begin")

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("Predict batch end")

In [19]:
train_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './humans/train',
        target_size=(image_width, image_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        './humans/validation',
        target_size=(image_width, image_height),
        batch_size=batch_size,
        class_mode='binary')

#model.save('catsndogs')

#validation_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255)
#validation_generator = validation_datagen.flow_from_dataframe(validate_df, "./data/validation/",  x_col='filename', y_col='category', target_size = (image_width, image_height), class_mode = 'categorical', batch_size = batch_size)

Found 2022 images belonging to 1 classes.
Found 2023 images belonging to 1 classes.


In [18]:
model.fit(
    train_generator,
        steps_per_epoch=2000,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800)

model.save('test')

Epoch 1/10


ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:630 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:620 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:952 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2292 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2651 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:613 run_step  **
        outputs = model.tr

In [ ]:
model = tensorflow.keras.models.load_model("test")

In [93]:
img = image.load_img('humans/validation/yes/01954.png', target_size=(150, 150))
x = image.img_to_array(img)
x = numpy.expand_dims(x, axis=0)
x = preprocess_input(x)

In [94]:
predict = model.predict(x)

In [95]:
print(predict)

[[1. 0.]]
